# Numerical Solutions to Partial Differential Equations

### Simple 3-d Heat Equation
$$
\nabla^2 \cdot u(x,y,z,t) = \frac{c \rho}{k} u_t
$$
### Elliptic Equations
Poisson Equation:
$$
u_{xx}(x,y) + u_{yy}(x,y) = f(x,y)
$$
Laplace's Equation:
$$
u_{xx}(x,y) + u_{yy}(x,y) = 0
$$
- If temperature within the region is determined by the temperature distribution on the boundary of the region, the constraints are called **Dirichlet boundary conditions** given by:
$$
u(x,y) = g(x,y)
$$
### Parabolic Equations
$$
u_t(x,y) - \alpha^2 u_{xx}(x,y) = 0
$$
- important in study of gas diffusion, also known as diffusion equation

### Hyperbolic Equations
- one-dimension wave equation is an exaqmple of a hyperbolic PDE
$$
\alpha^2 u_{xx}(x,t) - u_{tt}(x,t) =0
$$

### 12.1 Elliptic PDE
$$
\nabla^2 u(x,y) \equiv u_{xx}(x,y) + u_{yy}(x,y) = f(x,y)
$$
on $R = \{ (x,y) | a < x< b , c< y < d \}$ with $u(x,y) = g(x,y) \text{ for } (x,y) \in S$, where $S$ denotes the boundary of $R$. If $f$ and $g$ are continious on their domains, then there is a unique solution to this equation.


- method used is a 2-d adaption of the finite difference method for linear BVP.
First step is to choose integers $n$ and $m$ to define step sizes $h=(b-a)/n$ , $k=(d-c)/m$. Partition the interval $[a,b]$ into $n$ equal parts with width $h$ and interval $[c,d]$ into $m$ equal parts of width $k$.

- $x_i = a+ih$, $y_j = c+jk$

Using Taylor series in the variable $x$ about $x_i$, we can generate the centered-difference formula:
$$
u_{xx}(x_i,y_j) = \frac{1}{h^2} u(x_{i+1},y_j) - 2u(x_i,y_j)+u(x_{u-1},y_j) -\frac{h^2}{12}\frac{\partial^4 u}{\partial x^4}(\xi_i, y_j)
$$
and Taylor series in the variable $y$ about $y_j$ to generate:
$$
u_{xx}(x_i,y_j) = \frac{1}{k^2} u(x_{i},y_{j+1}) - 2u(x_i,y_j)+u(x_{u},y_{j-1}) -\frac{k^2}{12}\frac{\partial^4 u}{\partial y^4}(x_i, \eta_j)
$$
Poissons Eq at the points $(x_i,y_j)$ :

$$
\frac{1}{h^2} u(x_{i+1},y_j) - 2u(x_i,y_j)+u(x_{u-1},y_j) +\frac{1}{k^2} u(x_{i},y_{j+1}) - 2u(x_i,y_j)+u(x_{u},y_{j-1}) \\
= f(x_i,y_j) + \frac{k^2}{12}\frac{\partial^4 u}{\partial y^4}(x_i, \eta_j) + \frac{h^2}{12}\frac{\partial^4 u}{\partial x^4}(\xi_i, y_j)
$$
for each $i \in [1,n-1] \text{ and } j \in [1,m-1]$. The Boundary conditions are:
$$
u(x_0, y_j) = g(x_0, y_j) \text{ and } u(x_n, y_j) = g(x_n, y_j), \text{ for each } j \in [0,m] \\
u(x_i, y_0) = g(x_i, y_0) \text{ and } u(x_i, y_m) = g(x_i, y_m), \text{ for each } i \in [1,n-1]
$$

### Finite-Difference Method
$$
2  \left[\left(\frac{h}{k}\right)^2+1\right] w_{ij} -(w_{i+1,j}+w_{i-1,j}) - \left(\frac{h}{k}\right)^2(w_{i,j+1}+w_{i,j-1}) = -h^2f(x_i,y_j)
$$
for each $i\in[1,n-1] \text{ and } j \in [1,m-1]$ and

$$
w_{0,j} = g(x_0, y_j) \text{ and } w_{n,j} = g(x_n, y_j), \text{ for each } j \in [0,m] \\
w{i,0} = g(x_i, y_0) \text{ and } w{i,m} = g(x_i, y_m), \text{ for each } i \in [1,n-1]
$$

- This produces an $(n-1)(m-1) \times (n-1)(m-1)$ linear system with the unknowns being the approximations $w_{i,j}$ to $u(x_i,y_j)$ at the interior mesh points
- linear system is expressed for matrix calculations more efficiently if a relabeling of the interior mesh points is introduced:
$$
P_l = (x_i,y_j) \quad \text{ and } \quad w_l = w_i,j
$$
where $l=i+(m-1-j)(n-1), i,j\forall [1,n-1],[1,m-1]$

In [7]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
def PoissionFD(f,g,a,b,c,d,m,n,TOL,N):
    h=(b-a)/n
    k=(d-c)/m
    tx=np.arange(a,b+h,h)
    ty=np.arange(c,d+k,k)
    lam=h**2/k**2
    mu = 2*(1+lam)
    l = 1

    w = np.zeros(n,m)

    while l <= N: #Gauss-Seidel Iterations
        z = (-h**2 * f(tx[0],ty[m-1]) + g(a,ty[m-1]) + lam * g(tx[0],d) + lam* w[0,m-2] + w[1,m-1])/mu
        norm = np.abs(z-w[0,m-1])
        w[0,m-1] = z
        for i in range(1,n-3):
            z = (-h**2 * f(tx[i],y[m-1]) + lam*g[tx[i],d] + w[i-2,m-1] + w[i,m-1] + lam * w[i-1,m-2])/mu
            if np.abs(w[i-1,m-1] - z) > norm:
                norm = np.abs(w[i-1,m-1] - z)
            w[i-1,m-1] = z
            for j in range[m-2,2]: #wtf
                z = (-h**2*f(tx[0], ty[j]) + g[a,ty[j]] + lam*(w[0,j+1] + w[0,j-1]) + w[1,j] )/mu
                if np.abs(w[0,j] - z) > norm:
                    norm = np.abs(w[0,j] - z)
                w[0,j] = z
            for i in range(1,n-3):
                z = (-h**2*f(tx[i], ty[j]) + w[tx-2,j] + lam*w[i-1,j+1]  + w[i,j] + lam*w[i-1,j-1] )/mu
                if np.abs(w[i,j] - z) > norm:
                    norm = np.abs(w[i,j] - z)
                w[i,j] = z
            z = (-h**2 * f(tx[n-2], ty[j]) + g(b,ty[j]) + w[n-3,j] + lam*w[n-2,j+1]+lam*w[n-2,j-2])/mu
            if np.abs(w[n-2,j] - z) > norm:
                norm = np.abs(w[n-2,j]-z)
            w[n-2,j] = z
        z = (-h**2*f(tx[0],ty[0])+g(a,ty[0]) + lam*g[tx[0],c]+lam*w[0,1]+w[1,0])/mu
        if np.abs(w[0,0] - z) > norm:
            norm = np.abs(w[0,0]-z)
        w[0,0] = z 
        for i in range[1,n-3]: #step 15
            z = (-h**2 * f(tx[0,ty[0]]) + lam* g(tx[i-1], c)+w[i-2,1]+ lam*w[i-1,2]+w[i,1])/mu
            if np.abs(w[i-1,1]-z) > norm:
                norm = np.abs(w[i-1, 1 ]- z)
            w[i-1,1] = z
        z = (-h**2 * f(tx[n-2], ty[1])+g([b-1,ty[1]])+lam*g(tx[n-2],c)+w[n-3,1] + lam * w[n-2,2])/mu
        if np.abs(w[n-2,1]-z) > norm:
            norm = np.abs(w[n-2,1] - z)
        w[n-2,1] = z
        if norm <= TOL:
            return (tx,ty,w)
        l = l+1
    print("max iteration exceeded")




although the Gauss-Seidel iterative procedure is used for simplicity, it is advisable to use a direct technique such as Gaussian elimination when the system is small, on the order of 100 or less, because the positive definiteness ensures stability with respect to round-off errors

In [ ]:
def f(x,y):
    return x*np.e**y
def g(x,y):
    return 